### Notes

trying to do the yaml pyiceberg file kept failing, thus using the sqlcatalog to get it rolling

polars works best when you pass in the pyiceberg table object instead of trying to provide a metadata path; much cleaner

this demo takes duckdb to create a table, convert to pyarrow, push to iceberg, and read with polars :P

In [33]:
import duckdb
import polars as pl
from pyiceberg.catalog.sql import SqlCatalog

In [2]:
cn = duckdb.connect()

In [3]:
warehouse_path = "./icehouse"
catalog = SqlCatalog(
    "default",
    **{
        "uri": f"sqlite:///{warehouse_path}/icyhot.db",
        "warehouse": f"file://{warehouse_path}",
    },
)

In [4]:
catalog.create_namespace("dummy_data")

In [5]:
sql = """
select 1 as x
"""
df = cn.execute(sql).arrow()

In [ ]:
table = catalog.create_table(
    "dummy_data.ducks",
    schema=df.schema,
)

In [10]:
table.append(df)
len(table.scan().to_arrow())

3

In [11]:
cn.execute("""
    INSTALL iceberg;
    LOAD iceberg;
""")

In [ ]:
# duckdb chokes here
path = "~/dream_machine/iceberg/pyicy/icehouse/dummy_data.db/ducks/metadata/*.metadata.json"
sql = f"""
SELECT *
FROM iceberg_scan('{path}')
"""
cn.sql(sql).show()

In [37]:
import polars as pl
import uuid
from datetime import date
import numpy as np


In [61]:
#generate 1M rows of dummy data
rows = 1_000_000

df = pl.DataFrame({
    'row_id': pl.arange(0, rows, eager=True),
    'rpt_dt': pl.Series([date.today()] * rows),
    'some_val': pl.Series(np.floor(np.random.rand(rows) * 100).astype(int)),
    'txn_key': pl.Series([str(uuid.uuid4()) for _ in range(rows)])
}).to_arrow()


In [62]:
#create iceberg table of it
table = catalog.create_table(
    "dummy_data.dummy_ducks2",
    schema= df.schema,
)

In [67]:
table.append(df)

In [68]:
df2 = pl.scan_iceberg(table).collect()

In [69]:
df2.select(pl.col('row_id').count(), pl.col('some_val').sum().alias('blah'))

row_id,blah
u32,i64
3000000,148339965
